In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import numpy

import socketio
import asyncio

In [ ]:
sio = socketio.AsyncClient()
part2_model = tf.keras.models.load_model('models/part2')

In [ ]:
@sio.on('to part2', namespace="/net")
async def got_input(data):
    print("Received data in part2")
    inpt = tf.convert_to_tensor(data["server"])
    out = part2_model(inpt).numpy().tolist()
    await sio.emit('from node', data=("part2", out), namespace="/net")

@sio.on('connect', namespace="/net")
def connect():
    print("I'm connected!")

@sio.on('disconnect', namespace="/net")
def disconnect():
    print("I'm disconnected!")
    exit(0)

In [ ]:
await sio.connect('http://localhost:5000', namespaces=['/net'], auth={"name": "part2"})
await sio.wait()